In [1]:
%%capture
pip install overpy

In [2]:
import pandas as pd

file_path = 'comp_stores.csv'
comp_stores = pd.read_csv(file_path)

In [3]:
comp_stores.head()

,Pseudo Store ID,Latitude,Longitude,Visits,Chain
0,100000,37.0469,-93.3024,125014,Kum&Go
1,100001,37.0427,-93.2835,230643,Kum&Go
2,100002,37.0049,-93.2065,520220,Kum&Go
3,100003,37.0709,-93.2319,691043,Kum&Go
4,100004,35.9443,-95.6497,213666,Kum&Go


In [4]:
# comp_stores_sample = comp_stores.head()

In [ ]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Define radii in miles and their conversion to meters for the required radii only
radii_miles_to_meters = {
    1: 1609.34,
    3: 3 * 1609.34,
    5: 5 * 1609.34
}

# Define categories and their tags for required categories and radii only
required_categories_tags = {
    "Grocery_Stores_1mile": [
        ("amenity", "supermarket"),
        ("shop", "bakery"),
        ("shop", "convenience"),
        ("shop", "frozen_food"),
        ("shop", "department_store"),
        ("shop", "supermarket"),
        ("shop", "wholesale")
    ],
    "Food_Outlets_1mile": [
        ("amenity", "fast_food"),
        ("amenity", "food_court"),
        ("amenity", "restaurant"),
        ("amenity", "ice_cream"),
        ("shop", "cheese"),
        ("shop", "chocolate"),
        ("shop", "pasta"),
        ("shop", "pastry"),
        ("shop", "seafood"),
        ("shop", "food")
    ],
    "Educational_Institutions_1mile": [
        ("building", "college"),
        ("building", "school"),
        ("amenity", "library"),
        ("building", "university")
    ],
    "Residential_Housing_1mile": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "hut")
    ],
    "Commercial_Housing_1mile": [
        ("building", "commercial"),
        ("building", "office")
    ],
    "Fuel_Stations_Car_Wash_1mile": [
        ("amenity", "car_wash"),
        ("amenity", "charging_station"),
        ("amenity", "fuel")
    ],
    "Hospitals_3miles": [
        ("amenity", "clinic"),
        ("amenity", "nursing_home"),
        ("amenity", "pharmacy"),
        ("building", "hospital")
    ],
    "Apparels_3miles": [
        ("shop", "clothes"),
        ("shop", "fabric"),
        ("shop", "fashion")
    ],
    "Tourist_Destinations_5miles": [
        ("tourism", "attraction"),
        ("amenity", "events_venue"),
        ("amenity", "exhibition_centre"),
        ("boundary", "national_park"),
        ("boundary", "museum"),
        ("building", "beach_hut"),
        ("building", "castle")
    ],
    "Hotels_5miles": [
        ("building", "hotel")
    ],
    "Airports_5miles": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
        ("aeroway", "aerodrome")  # Added to include the main airport tag
    ]
}

# Define a delay duration in seconds
delay_duration = 0.5

def fetch_feature_count(api, lat, lon, required_categories_tags):
    category_counts = {}
    
    for category, tags in required_categories_tags.items():
        suffix = category.split("_")[-1]  # Extract the mile part from the category name
        radius = radii_miles_to_meters[int(suffix[0])]  # Extract the radius in miles and convert to meters
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        feature_counts = fetch_feature_count(api, lat, lon, required_categories_tags)
        feature_counts.update({
            "Latitude": lat,
            "Longitude": lon
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    final_df = pd.DataFrame(all_feature_counts)

    # Display and save the DataFrame
    print(final_df)
    final_df.to_csv("comp_nearby_places.csv", index=False)
    print("Competition data collection complete. CSV file created.")

if __name__ == "__main__":
    # Assuming meijer_stores is defined elsewhere with 'Latitude' and 'Longitude' columns
    main(comp_stores)

Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 2
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 7
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institutions_1mile. Count: 0
Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 0
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 2
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 1
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 1
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 0
Fetching data for category: Tourist_Destinations_5miles
Complete

Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 3
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 2
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 0
Fetching data for category: Tourist_Destinations_5miles
Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 0
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 3
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 9
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 23
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institu

Completed fetching for category: Airports_5miles. Count: 4
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 0
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 0
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institutions_1mile. Count: 0
Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 0
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 0
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 1
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 4
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 0
Fetch

Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 3
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 3
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 5
Fetching data for category: Tourist_Destinations_5miles
Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 1
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 6
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 4
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 2
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institut

Completed fetching for category: Airports_5miles. Count: 3
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 4
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 9
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institutions_1mile. Count: 5
Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 4
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 37
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 2
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 16
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 2
Fet

...

Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 8
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 6
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 9
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 7
Fetching data for category: Tourist_Destinations_5miles
Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 1
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 5
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 5
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. 

Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 9
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 5
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 2
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 0
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institutions_1mile. Count: 0
Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 3
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 1
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 2
Fetching data for category: Hospitals_3miles
Completed fetching for ca

Completed fetching for category: Residential_Housing_1mile. Count: 31
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 72
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 9
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 3
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 2
Fetching data for category: Tourist_Destinations_5miles
Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 0
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 2
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 11
Fetching data for category: 

Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 0
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 3
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_Stores_1mile. Count: 2
Fetching data for category: Food_Outlets_1mile
Completed fetching for category: Food_Outlets_1mile. Count: 5
Fetching data for category: Educational_Institutions_1mile
Completed fetching for category: Educational_Institutions_1mile. Count: 0
Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 0
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 0
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 3

Fetching data for category: Residential_Housing_1mile
Completed fetching for category: Residential_Housing_1mile. Count: 0
Fetching data for category: Commercial_Housing_1mile
Completed fetching for category: Commercial_Housing_1mile. Count: 0
Fetching data for category: Fuel_Stations_Car_Wash_1mile
Completed fetching for category: Fuel_Stations_Car_Wash_1mile. Count: 4
Fetching data for category: Hospitals_3miles
Completed fetching for category: Hospitals_3miles. Count: 4
Fetching data for category: Apparels_3miles
Completed fetching for category: Apparels_3miles. Count: 0
Fetching data for category: Tourist_Destinations_5miles
Completed fetching for category: Tourist_Destinations_5miles. Count: 0
Fetching data for category: Hotels_5miles
Completed fetching for category: Hotels_5miles. Count: 0
Fetching data for category: Airports_5miles
Completed fetching for category: Airports_5miles. Count: 2
Fetching data for category: Grocery_Stores_1mile
Completed fetching for category: Grocery_

In [ ]:
file_path = 'comp_nearby_places.csv'
comp_all_stores_nearby_places = pd.read_csv(file_path)
comp_all_stores_nearby_places.head()

In [ ]:
# Rename 'Latitude' and 'Longitude' columns in meijer_stores DataFrame to 'latitude' and 'longitude'
comp_stores.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)

# Now perform the merge
final_comp_df = pd.merge(comp_stores, comp_all_stores_nearby_places, on=['latitude','longitude'], how='inner')
final_comp_df.head()

final_comp_df = pd.merge(comp_stores, comp_all_stores_nearby_places, on=['latitude','longitude'], how='inner')
final_comp_df.head()

In [ ]:
final_comp_df.to_csv("final_comp_df.csv", index=False)

In [ ]:
final_comp_df.head()